In [1]:
import pandas as pd
import torch
import numpy as np
import pickle
import os
from tqdm import tqdm
os.getcwd()

'/n/holyscratch01/shakhnovich_lab/thomasloux/shakhnovitch-research/ML_Kd_ESM3'

In [ ]:
with open("embedding_Desai_15loci_emb_esm2_650m.pkl", "rb") as f:
    esm2_small = pickle.load(f)

In [ ]:
sequences = np.load("df_Desai_15loci_sequence.npy", allow_pickle=True)

In [13]:
esm2_small[10000]

{'label': '10000',
 'sequence': 'NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNLASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGNIADYNYKLPDDFTGCVIAWNSNKLDSKVSGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLRSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKST',
 'cls_representation': [0.041843872517347336,
  0.024716759100556374,
  0.021954035386443138,
  0.048607442528009415,
  -0.03146258369088173,
  0.028565462678670883,
  0.07821973413228989,
  -0.053690049797296524,
  -0.10084676742553711,
  0.005065106321126223,
  0.11444398015737534,
  0.06100589036941528,
  0.11115311086177826,
  0.04945193976163864,
  0.17510229349136353,
  0.05585668608546257,
  0.0031550845596939325,
  -0.03838125243782997,
  0.011269060894846916,
  -0.103180430829525,
  -0.06872673332691193,
  -0.1204887181520462,
  0.05709877237677574,
  0.20997443795204163,
  -0.09009072929620743,
  0.011048154905438423,
  0.018512895330786705,
  0.022742722183465958,
  0.02234356291592121,
  -0.005024975631386042,
  0.09883879870176315,
  0.0581

In [34]:
len(esm2_small)

32768

In [18]:
sequences[0]

'NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSKPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKST'

In [6]:
mutations = """
G339D
S371L
S373P
S375F
K417N
N440K
G446S
S477N
T478K
E484A
Q493R
G496S
Q498R
N501Y
Y505H
"""
index_depart = 331
mutations = {int(m[1:4]) - index_depart: [m[0], m[4]] for m in mutations.split("\n") if m}

In [56]:
mutations

{8: ['G', 'D'],
 40: ['S', 'L'],
 42: ['S', 'P'],
 44: ['S', 'F'],
 86: ['K', 'N'],
 109: ['N', 'K'],
 115: ['G', 'S'],
 146: ['S', 'N'],
 147: ['T', 'K'],
 153: ['E', 'A'],
 162: ['Q', 'R'],
 165: ['G', 'S'],
 167: ['Q', 'R'],
 170: ['N', 'Y'],
 174: ['Y', 'H']}

In [7]:
def get_one_hot_from_sequence(sequence, mutations_possible):
    return [int(aa[1]==sequence[index]) for index, aa in mutations_possible.items()]

In [78]:
get_one_hot_from_sequence(esm2_small[100]["sequence"], mutations)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]

In [80]:
embeddings_in_sequence_order = []
sequences_to_check = []
for seq in tqdm(sequences):
    one_hot = get_one_hot_from_sequence(seq, mutations)
    index = sum([2**i for i, x in enumerate(one_hot) if x])
    embeddings_in_sequence_order.append(esm2_small[index]["mean_representation"])
    sequences_to_check.append(seq)

  0%|                                                                                                                     | 0/32768 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 32768/32768 [00:00<00:00, 280666.94it/s]


In [ ]:
embeddings_in_sequence_order = torch.tensor(embeddings_in_sequence_order)
torch.save(embeddings_in_sequence_order, "embeddings_sequnce_esm2_650m.pt")

In [81]:
assert all([a == b for a, b in zip(sequences, sequences_to_check)])

In [ ]:
with open("embedding_Desai_15loci_emb_esm2_15b.pkl", "rb") as f:
    esm2 = pickle.load(f)

In [ ]:
embeddings_in_sequence_order = []
sequences_to_check = []
for seq in tqdm(sequences):
    one_hot = get_one_hot_from_sequence(seq, mutations)
    index = sum([2**i for i, x in enumerate(one_hot) if x])
    embeddings_in_sequence_order.append(esm2[index]["mean_representation"])
    sequences_to_check.append(seq)

assert all([a == b for a, b in zip(sequences, sequences_to_check)])

embeddings_in_sequence_order = torch.tensor(embeddings_in_sequence_order)
torch.save(embeddings_in_sequence_order, "embeddings_sequnce_esm2_15b.pt")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 32768/32768 [00:00<00:00, 276302.52it/s]


In [ ]:
with open("embedding_Desai_15loci_emb_esm1v_650m.pkl", "rb") as f:
    esm1v = pickle.load(f)

In [ ]:
embeddings_in_sequence_order = []
sequences_to_check = []
for seq in tqdm(sequences):
    one_hot = get_one_hot_from_sequence(seq, mutations)
    index = sum([2**i for i, x in enumerate(one_hot) if x])
    embeddings_in_sequence_order.append(esm1v[index]["mean_representation"])
    sequences_to_check.append(seq)

assert all([a == b for a, b in zip(sequences, sequences_to_check)])

embeddings_in_sequence_order = torch.tensor(embeddings_in_sequence_order)
torch.save(embeddings_in_sequence_order, "embeddings_sequnce_esm1v_650m.pt")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 32768/32768 [00:00<00:00, 287973.80it/s]


In [ ]:
# export one-hot encoding 
one_hot = [get_one_hot_from_sequence(seq, mutations) for seq in sequences]
one_hot = torch.tensor(one_hot)
torch.save(one_hot, "embeddings_sequence_one_hot.pt")